In [ ]:
from knuckledragger import *
# Peano Arithmetic

# Z3py adt of natural numbers
Nat = Datatype("Nat")
Nat.declare("zero")
Nat.declare("succ", ("pred", Nat))
Nat = Nat.create()
print(Nat.succ(Nat.zero))

# Peano
def induct(P : Form) -> Thm:
    print(P.sort())
    assert P.sort().name() == "Array"
    assert P.sort().domain() == Nat
    assert P.sort().range() == BoolSort()
    n = FreshConst(Nat)
    hyp = P[Nat.zero] & ForAll([n], P[n] > P[Nat.succ(n)])
    #------------------------------------------
    conc =  ForAll([n], P[n])
    return trust(hyp > conc)

x,y = Consts("x y", Nat)
add = Function("add", Nat, Nat, Nat)
zero_add = trust(ForAll([x], add(Nat.zero, x) == x))
succ_add = trust(ForAll([x,y], add(Nat.succ(x), y) == Nat.succ(add(x, y))))

P = Lambda([x], add(x,Nat.zero) == add(Nat.zero,x))

base = modus([], P[Nat.zero])
ind = modus([zero_add, succ_add], ForAll([x], P[x] > P[Nat.succ(x)]))
add_zero = modus([base,ind,induct(P)], ForAll([x], P[x]))
add_zero_prime = modus([zero_add, succ_add, induct(P)], ForAll([x], P[x]))


P = Lambda([x], ForAll([y], add(x,Nat.succ(y)) == Nat.succ(add(x,y))))
add_succ = modus([zero_add,succ_add, induct(P)], ForAll([x],P[x]))
comm_add = modus([zero_add, succ_add, add_zero, add_succ, induct(Lambda([x], ForAll([y], add(x,y) == add(y,x))))] ,
                  ForAll([x,y], add(x, y) == add(y, x)))

comm_add

def induct_int(P : Form) -> Thm:
    assert P.sort().name() == "Array"
    assert P.sort().domain() == IntSort()
    assert P.sort().range() == BoolSort()
    n = FreshConst(IntSort())
    hyp = P[0] & ForAll([n], P[n] > P[n - 1] & P[n + 1])
    #------------------------------------------
    conc =  ForAll([n], P[n])
    return trust(hyp > conc)
# https://math.stackexchange.com/questions/2659184/can-induction-be-done-to-prove-statements-for-integers


def induct_posint(P : Form) -> Thm:
    assert P.sort().name() == "Array"
    assert P.sort().domain() == IntSort()
    assert P.sort().range() == BoolSort()
    n = FreshConst(IntSort())
    hyp = P[0] & ForAll([n], (P[n] & n >= 0) > P[n + 1])
    #------------------------------------------
    conc =  ForAll([n], (n >= 0) > P[n])
    return trust(hyp > conc)
# extending with injection axioms into the Ints

def induct(P):
  # assert P.type == Nat -> Bool ?
  n = FreshConst(Nat)
  return ForAll(n, Implies(P(Nat.zero), Forall(n, Implies(P(n), P(Nat.succ(n))))), Forall(n, P(n)))

inj = Function("inj", Nat, IntSort())
n = FreshConst(Nat)
axioms = [
  inj(Nat.zero) == 0,
  ForAll(n, inj(Nat.succ(n)) == inj(n) + 1) # recursive definition of inj
]

theorem1 = ForAll(n, inj(n) >= 0)
theorem2 = ForAll(i, Implies(i >= 0, Exist(n, inj(n) == i)))
P = lambda x: 
x,y,n = Ints("x y n")
even, even_def = define("even", Lambda([x], Exists([y], x == y + y)))
odd, odd_def = define("odd", Lambda([x], Exists([y], x == y + y + 1)))

even0 = infer([even_def], even[0])
#ind = infer([even_def,odd_def], QForAll([n], even[n], odd[n+1]))
ind1 = infer([even_def,odd_def], ForAll([n], even[n] == odd[n+1]))
ind2 = infer([even_def,odd_def], ForAll([n], odd[n] == even[n+1]))
#ind = infer([even_def,odd_def, ind], QForAll([n], odd[n-1], even[n]))
#ind2 = infer([even_def,odd_def], QForAll([n], odd[n], Exists([y], n + 1 == (y + 1) + (y + 1))))
#ind3 = infer([even_def, odd_def, ind2], QForAll([n], odd[n], Exists([y], n + 1 == y + y)))
#even_or_odd = infer([even_def, odd_def], ForAll([n], even[n] | odd[n]))

# proof relevant predicates. The evidence is unsealed
# Can't really do a depndnt product / sum type easily.
# fst(p)
x,y,n,m = Ints("x y n m")
even_ev, even_ev_def = define("even", Lambda([x,y], x == y + y))
odd_ev, odd_ev_def = define("odd", Lambda([x,y], x == y + y + 1))

even0 = infer([even_ev_def], even_ev[0,0])
ind1 = infer([even_ev_def,odd_ev_def], ForAll([n,m], even_ev[n,m] == odd_ev[n+1,m]))
ind2 = infer([even_ev_def,odd_ev_def], ForAll([n,m], odd_ev[n,m] == even_ev[n+1,m+1]))

#even_ev_remove = infer([even_ev_def], QForAll([n,m], Exists([m], even_ev[n,m])), even[n])
# make definition the existential of even_ev
# even, even_def = define("even", Lambda([x], Exists([y], even_ev[x,y])))

# sklolem version
# Implies(even[n], even_ev[n,half[n]]
# Implies(odd[n], odd_ev[n,half[n] + 1]
half, half_def = define("half", Lambda([x], x / 2))
infer([half_def, even_ev_def, odd_ev_def], ForAll([n], even_ev[n, half[n]] | odd_ev[n, half[n]]))

